# Загрузка данных

In [928]:
import pandas as pd
import json

In [929]:
FILE_NAME='data10-04.xlsx'

In [930]:
df = pd.read_excel(f'datasets/{FILE_NAME}')

In [931]:
df.head()

,F_User,C_Record_ID,C_Table,N_Operation,C_Json,D_Date_Created
0,64b18dc9-69d0-4286-887f-7469ebc1b47d,9770f581-b89f-4fb3-94f5-93173bb85ad1,ED_Meter_Readings-ListView,1,"{""LINK"":""50fb47a1-6711-4e65-9232-3700daaa58bd""...",2023-04-09 16:42:47.637
1,64b18dc9-69d0-4286-887f-7469ebc1b47d,5f97a8be-b3ad-4982-9191-fbed301b3c69,DD_Doc_Details-ListView,1,"{""LINK"":""a95caaa0-a674-4f31-84d4-cbb52f0317f1""...",2023-04-09 16:42:47.980
2,64b18dc9-69d0-4286-887f-7469ebc1b47d,d4c614dd-3114-4326-8ddd-bb9d74382b03,DD_Docs_Out-ListView,0,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:48.627
3,64b18dc9-69d0-4286-887f-7469ebc1b47d,9ee02cde-4a6f-4c51-8e3b-b14ddae38bee,DD_Files-ListView,0,"{""LINK"":""e010d6d0-1df5-790d-19fe-b007d7be4207""...",2023-04-09 16:42:49.557
4,64b18dc9-69d0-4286-887f-7469ebc1b47d,6829a2cd-007c-45d8-88ca-e143617555c4,DD_Docs-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:50.010


In [932]:
df['D_Date_Created'] = pd.to_datetime(df['D_Date_Created'])
df['N_Operation'] = df['N_Operation'].astype('int8')

In [933]:
df.drop_duplicates(inplace=True)

In [934]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29561 entries, 0 to 31944
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   F_User          29561 non-null  object        
 1   C_Record_ID     29561 non-null  object        
 2   C_Table         29561 non-null  object        
 3   N_Operation     29561 non-null  int8          
 4   C_Json          29216 non-null  object        
 5   D_Date_Created  29561 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int8(1), object(4)
memory usage: 1.4+ MB


# Алгоритмы

In [935]:
users = df['F_User'].unique()

In [936]:
use_records = []
correct_records = []

## Алгоритм

Последовательность ED_Meter_Readings-ListView -> DD_Doc_Details-ListView -> DD_Docs_Out-ListView -> DD_Files-ListView -> DD_Docs-ListView

In [937]:
dict_algoritm_1 = {}
for i in range(len(users)):
    dict_algoritm_1[users[i]] = []

In [938]:
def algoritm_1(user_id):    
    items = df.loc[df['F_User'] == user_id].sort_values(by=['D_Date_Created'], ascending=True)

    indexes = list(items.index)
    tables = list(items['C_Table'])
    operations = list(items['N_Operation'])
    json_items = list(items['C_Json'])
    resords = list(items['C_Record_ID'])

    buffer = {
        'ED_Meter_Readings-ListView': [],
        'DD_Doc_Details-ListView': [],
        'DD_Docs_Out-ListView': [],
        'DD_Files-ListView': [],
        'DD_Docs-ListView': [],
        'Other': []
    }
    
    for i in range(len(tables)):
        table = tables[i]
        operation = operations[i]
        json_item = json_items[i]
        index = indexes[i]
        record = resords[i]

        if table == 'ED_Meter_Readings-ListView':
            buffer['ED_Meter_Readings-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Doc_Details-ListView':
            buffer['DD_Doc_Details-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Docs_Out-ListView':
            buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Files-ListView':
            try:
                buffer['DD_Files-ListView'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['DD_Files-ListView'].append([index, record, table, operation, None])
            
        if table == 'DD_Docs-ListView':
            buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
            if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                for key in buffer:
                    for j in range(len(buffer[key])):
                        if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
                            dict_algoritm_1[user_id].append(buffer[key][j])
                        else:
                            correct_records.append(buffer[key][j][1])
                            
                        use_records.append(buffer[key][j][1])
                        
                        df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]
                        
            buffer = {
                'ED_Meter_Readings-ListView': [],
                'DD_Doc_Details-ListView': [],
                'DD_Docs_Out-ListView': [],
                'DD_Files-ListView': [],
                'DD_Docs-ListView': [],
                'Other': []
            }
            
        if table not in (['ED_Meter_Readings-ListView', 'DD_Doc_Details-ListView', 'DD_Docs_Out-ListView', 'DD_Files-ListView', 'DD_Docs-ListView']):
            try:
                buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['Other'].append([index, record, table, operation, None])
            
    # дозаписываем данные
    if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
        for key in buffer:
            for j in range(len(buffer[key])):
                dict_algoritm_1[user_id].append(buffer[key][j])
                use_records.append(buffer[key][j][1])
                
                df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]

In [939]:
# use_records = []
# correct_records = []
# dict_algoritm_1 = {'7d865ee6-9803-427f-81f4-c8f956f6b46b': []}
# algoritm_1('7d865ee6-9803-427f-81f4-c8f956f6b46b')

# df['Use'] = df['C_Record_ID'].isin(use_records)

# display(df.loc[df['F_User'] == '7d865ee6-9803-427f-81f4-c8f956f6b46b', ['C_Record_ID', 'C_Table', 'D_Date_Created', 'Group_ID', 'Use']].sort_values(by=['D_Date_Created'], ascending=True).head(50))

In [940]:
for i in range(len(users)):
    user_id = users[i]
    algoritm_1(user_id)

In [941]:
df['Use'] = df['C_Record_ID'].isin(use_records)
print(f'Обработано записей: {len(use_records)}')

Обработано записей: 25165


In [942]:
df.to_excel('outputs/algoritm_1.xlsx')

## Алгоритм

Последовательность ED_Meter_Readings-ListView -> DD_Docs_Out-ListView -> DD_Files-ListView -> DD_Docs-ListView

In [943]:
# dict_algoritm_2 = {}
# for i in range(len(users)):
#     dict_algoritm_2[users[i]] = []

In [944]:
# def algoritm_2(user_id):    
#     items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

#     indexes = list(items.index)
#     tables = list(items['C_Table'])
#     operations = list(items['N_Operation'])
#     json_items = list(items['C_Json'])
#     resords = list(items['C_Record_ID'])

#     buffer = {
#         'ED_Meter_Readings-ListView': [],
#         'DD_Docs_Out-ListView': [],
#         'DD_Files-ListView': [],
#         'DD_Docs-ListView': [],
#         'Other': []
#     }
    
#     is_meter = False
#     is_not_meter = False
    
#     for i in range(len(tables)):
#         table = tables[i]
#         operation = operations[i]
#         json_item = json_items[i]
#         index = indexes[i]
#         record = resords[i]

#         if table == 'ED_Meter_Readings-ListView':
#             is_meter = True
            
#             if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                 
#                 for key in buffer:
#                     for j in range(len(buffer[key])):
#                         if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
#                             dict_algoritm_2[user_id].append(buffer[key][j])
#                         else:
#                             correct_records.append(buffer[key][j][1])
                            
#                         use_records.append(buffer[key][j][1])
                            
#             if is_not_meter == True:   
#                 buffer = {
#                     'ED_Meter_Readings-ListView': [],
#                     'DD_Docs_Out-ListView': [],
#                     'DD_Files-ListView': [],
#                     'DD_Docs-ListView': [],
#                     'Other': []
#                 }
                
#             buffer['ED_Meter_Readings-ListView'].append([index, record, table, operation, json.loads(json_item)])
                
#             is_not_meter = False
#         else:
#             is_meter = False
            
#         if table == 'DD_Docs_Out-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0:
#             buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
#         if table == 'DD_Files-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0:
#             try:
#                 buffer['DD_Files-ListView'].append([index, record, table, operation, json.loads(json_item)])
#             except:
#                 buffer['DD_Files-ListView'].append([index, record, table, operation, None])
            
#         if table == 'DD_Docs-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0:
#             buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
#             is_not_meter = True
            
#         if table not in (['ED_Meter_Readings-ListView', 'DD_Docs_Out-ListView', 'DD_Files-ListView', 'DD_Docs-ListView']):
#             try:
#                 buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
#             except:
#                 buffer['Other'].append([index, record, table, operation, None])
            
#     # дозаписываем данные
#     if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
#         for key in buffer:
#             for j in range(len(buffer[key])):
#                 dict_algoritm_2[user_id].append(buffer[key][j])
#                 use_records.append(buffer[key][j][1])

In [945]:
# for i in range(len(users)):
#     user_id = users[i]
#     algoritm_2(user_id)

In [946]:
# df['Use'] = df['C_Record_ID'].isin(use_records)
# print(f'Обработано записей: {len(use_records)}')

In [947]:
# df.to_excel('outputs/algoritm_2.xlsx')

## Алгоритм

Последовательность ED_Meter_Readings-ListView -> DD_Doc_Details-ListView -> DD_Docs_Out-ListView -> DD_Docs-ListView

In [948]:
# dict_algoritm_3 = {}
# for i in range(len(users)):
#     dict_algoritm_3[users[i]] = []

In [949]:
# def algoritm_3(user_id):    
#     items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

#     indexes = list(items.index)
#     tables = list(items['C_Table'])
#     operations = list(items['N_Operation'])
#     json_items = list(items['C_Json'])
#     resords = list(items['C_Record_ID'])

#     buffer = {
#         'ED_Meter_Readings-ListView': [],
#         'DD_Doc_Details-ListView': [],
#         'DD_Docs_Out-ListView': [],
#         'DD_Docs-ListView': [],
#         'Other': []
#     }
    
#     is_meter = False
#     is_not_meter = False
    
#     for i in range(len(tables)):
#         table = tables[i]
#         operation = operations[i]
#         json_item = json_items[i]
#         index = indexes[i]
#         record = resords[i]

#         if table == 'ED_Meter_Readings-ListView':
#             is_meter = True
            
#             if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                
#                 for key in buffer:
#                     for j in range(len(buffer[key])):
#                         if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
#                             dict_algoritm_3[user_id].append(buffer[key][j])
#                         else:
#                             correct_records.append(buffer[key][j][1])
                            
#                         use_records.append(buffer[key][j][1])
                            
#             if is_not_meter == True:   
#                 buffer = {
#                     'ED_Meter_Readings-ListView': [],
#                     'DD_Doc_Details-ListView': [],
#                     'DD_Docs_Out-ListView': [],
#                     'DD_Docs-ListView': [],
#                     'Other': []
#                 }
                
#             buffer['ED_Meter_Readings-ListView'].append([index, record, table, operation, json.loads(json_item)])
                
#             is_not_meter = False
#         else:
#             is_meter = False
            
#         if table == 'DD_Doc_Details-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0:
#             buffer['DD_Doc_Details-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
#         if table == 'DD_Docs_Out-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0:
#             buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
#         if table == 'DD_Docs-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0:
#             buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
#             is_not_meter = True
        
#         if table not in (['ED_Meter_Readings-ListView', 'DD_Doc_Details-ListView', 'DD_Docs_Out-ListView', 'DD_Docs-ListView']):
#             try:
#                 buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
#             except:
#                 buffer['Other'].append([index, record, table, operation, None])
            
#     # дозаписываем данные
#     if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
#         for key in buffer:
#             for j in range(len(buffer[key])):
#                 dict_algoritm_3[user_id].append(buffer[key][j])
#                 use_records.append(buffer[key][j][1])

In [950]:
# for i in range(len(users)):
#     user_id = users[i]
#     algoritm_3(user_id)

In [951]:
# df['Use'] = df['C_Record_ID'].isin(use_records)
# print(f'Обработано записей: {len(use_records)}')

In [952]:
# df.to_excel('outputs/algoritm_3.xlsx')

## Алгоритм

Последовательность DD_Doc_Details-ListView -> DD_Docs_Out-ListView -> DD_Docs-ListView

In [953]:
dict_algoritm_4 = {}
for i in range(len(users)):
    dict_algoritm_4[users[i]] = []

In [954]:
def algoritm_4(user_id):    
    items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

    indexes = list(items.index)
    tables = list(items['C_Table'])
    operations = list(items['N_Operation'])
    json_items = list(items['C_Json'])
    resords = list(items['C_Record_ID'])

    buffer = {
        'DD_Doc_Details-ListView': [],
        'DD_Docs_Out-ListView': [],
        'DD_Docs-ListView': [],
        'Other': []
    }
    
    for i in range(len(tables)):
        table = tables[i]
        operation = operations[i]
        json_item = json_items[i]
        index = indexes[i]
        record = resords[i]

        if table == 'DD_Doc_Details-ListView': 
            buffer['DD_Doc_Details-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Docs_Out-ListView':
            buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Docs-ListView':
            buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
            if len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                for key in buffer:
                    for j in range(len(buffer[key])):
                        if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
                            dict_algoritm_4[user_id].append(buffer[key][j])
                        else:
                            correct_records.append(buffer[key][j][1])

                        use_records.append(buffer[key][j][1])

                        df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]
                            
            buffer = {
                'DD_Doc_Details-ListView': [],
                'DD_Docs_Out-ListView': [],
                'DD_Docs-ListView': [],
                'Other': []
            }
        
        if table not in (['DD_Doc_Details-ListView', 'DD_Docs_Out-ListView', 'DD_Docs-ListView']):
            try:
                buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['Other'].append([index, record, table, operation, None])
            
    # дозаписываем данные
    if len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
        for key in buffer:
            for j in range(len(buffer[key])):
                dict_algoritm_4[user_id].append(buffer[key][j])
                use_records.append(buffer[key][j][1])
                
                df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]

In [955]:
for i in range(len(users)):
    user_id = users[i]
    algoritm_4(user_id)

In [956]:
df['Use'] = df['C_Record_ID'].isin(use_records)
print(f'Обработано записей: {len(use_records)}')

Обработано записей: 28304


In [957]:
df.to_excel('outputs/algoritm_4.xlsx')

## Алгоритм

Последовательность DD_Docs_Out-ListView -> DD_Files-ListView -> DD_Docs-ListView

In [958]:
dict_algoritm_5 = {}
for i in range(len(users)):
    dict_algoritm_5[users[i]] = []

In [959]:
def algoritm_5(user_id):    
    items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

    indexes = list(items.index)
    tables = list(items['C_Table'])
    operations = list(items['N_Operation'])
    json_items = list(items['C_Json'])
    resords = list(items['C_Record_ID'])

    buffer = {
        'DD_Docs_Out-ListView': [],
        'DD_Files-ListView': [],
        'DD_Docs-ListView': [],
        'Other': []
    }
    
    for i in range(len(tables)):
        table = tables[i]
        operation = operations[i]
        json_item = json_items[i]
        index = indexes[i]
        record = resords[i]

        if table == 'DD_Docs_Out-ListView':
            buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Files-ListView':
            try:
                buffer['DD_Files-ListView'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['DD_Files-ListView'].append([index, record, table, operation, None])
            
        if table == 'DD_Docs-ListView':
            buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
            if len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                for key in buffer:
                    for j in range(len(buffer[key])):
                        if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
                            dict_algoritm_5[user_id].append(buffer[key][j])
                        else:
                            correct_records.append(buffer[key][j][1])
                            
                        use_records.append(buffer[key][j][1])
                        
                        df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]
            buffer = {
                'DD_Docs_Out-ListView': [],
                'DD_Files-ListView': [],
                'DD_Docs-ListView': [],
                'Other': []
            }
        
        if table not in (['DD_Docs_Out-ListView', 'DD_Files-ListView', 'DD_Docs-ListView']):
            try:
                buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['Other'].append([index, record, table, operation, None])
            
    # дозаписываем данные
    if len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
        for key in buffer:
            for j in range(len(buffer[key])):
                dict_algoritm_5[user_id].append(buffer[key][j])
                use_records.append(buffer[key][j][1])
                
                df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]

In [960]:
for i in range(len(users)):
    user_id = users[i]
    algoritm_5(user_id)

In [961]:
df['Use'] = df['C_Record_ID'].isin(use_records)
print(f'Обработано записей: {len(use_records)}')

Обработано записей: 28846


In [962]:
df.to_excel('outputs/algoritm_5.xlsx')

In [963]:
df['Group_ID'].nunique()

4611